In [ ]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')

from fanc_seg_utils.annotation_interface import neuroglancer_utilities,schema_download,schema_upload,statemanager
from fanc_seg_utils.skeletons import catmaid_utilities,skeleton_manipulations,skeletonization
import pandas as pd
import numpy as np
import nglui 
import pymaid
from concurrent import futures
from scipy import stats
from scipy.stats import norm
from cloudvolume import CloudVolume
from pathlib import Path
import json
from meshparty import trimesh_vtk as vtk
from annotationframeworkclient import FrameworkClient

In [2]:
client,tokens = neuroglancer_utilities.get_client()
cv = CloudVolume(neuroglancer_utilities.get_cv_path('Dynamic_V1')['url'],use_https=True,secrets=neuroglancer_utilities.get_token())

In [3]:
sm = statemanager.StateManager()

             state_id                                       description
0  261002187096550146                                    MNs,10Bs,Claws
1  838783366227209333  Slow Tibia Flexor in V1 chunkedgraph and V2 flat
2  838783366227209333  Slow Tibia Flexor in V1 chunkedgraph and V2 flat
3  585521373111957130              Comparison of V1 and V2 chunkedgraph
4  758492429940665723                                     9A_T1 neurons
5  299984539931822600                                              PMNs
6  336757619423424673                          81A07 synapse annotation
7  769530493626036401                                     Chen2020 data
8  837109578740814693                           9a_beta-club annotation


In [4]:
state = client.state.get_state_json(sm.df.state_id[7])
nglui.parser.get_layer(state,'13B')

{'annotationColor': '#0037ff',
 'annotationSelectionShowsSegmentation': False,
 'annotationTags': [],
 'annotations': [{'description': '13B_alpha1_T1R',
   'id': 'e0afc456d36e0ffcccdc26a561276e1c68d80b5c',
   'point': [41119.5625, 93109.8125, 3264.537353515625],
   'tagIds': [],
   'type': 'point'},
  {'description': '13B_alpha2_T1R',
   'id': '4adc357feb9bd75b3c54b6425f4f319600b13fc5',
   'point': [44111.78125, 89631.3359375, 3265.533447265625],
   'tagIds': [],
   'type': 'point'},
  {'description': '13B_alpha3_T1R',
   'id': '859995debb314724477f02a2b7ae29a1bfbe6223',
   'point': [44830.7578125, 90352.8828125, 3262.0146484375],
   'tagIds': [],
   'type': 'point'},
  {'description': '13B_alpha4_T1R',
   'id': 'ad83bf47e082b64a978c0b8c850cc32253cde9ed',
   'point': [44813.73828125, 91078.640625, 3265.327392578125],
   'tagIds': [],
   'type': 'point'},
  {'description': '13B_alpha5_T1R',
   'id': '46ce8af1b54126fb673ef454a8bd840ea39b498a',
   'point': [45531.21484375, 90928.9453125, 

In [10]:
def soma_table_entries(state,layer_name = 'cells'):
    ''' Generate entries for a soma table using the bound_tag schema 
    Args:
    state: json, json state from get_json_state
    layer_name: name of layer containing soma coords
    ### Change this to points later. This is just for database copying because I used a stupid annotation
    
    Returns:
    entries: list, list of dict entries for the bound_tag schema'''
    
    cell_layer = nglui.parser.get_layer(state,layer_name)
    entries = []
    for i in cell_layer['annotations']:
        entry = {'tag': i['description'],
                 'pt': {'position': i['point']},
                 'superceded_id':[111111,222222]}
        entries.append(entry)
    return(entries)
        

def upload_cells(client,cell_entries,table_name,description=None):
    ''' Upload cell entries to a soma dable using the bound_tag schema'
    Args:
    client: annotation client object
    cell_entries: list, list of dicts from soma_table_entries
    table_name: str, table name to create. If it exists, it will populate the existing one. 
    description: str, description to supply if creating a new table. It will fail if you don't provide one. 
    
    ##TODO: Some sort of check for redundant synapses is necessary. We could make each upload need to go to a new table, but that seems absurd. 
'''
    
    try:
        client.annotation.create_table(table_name=table_name,
                                   schema_name='bound_tag',
                                   description=description)
    except:
        print('table exists')
        
    for i in cell_entries:
        try:
            client.annotation.post_annotation(table_name=table_name, data=[i])
        except:
            print('Fail',entry)



def get_synapses(state,synapse_layer='synapses'):
    ''' Get the synapse coordinates from a json state and return pre,post,center.
    Args:
        state: json,  json state from get_state_json 
        synapse_layer: str, name of layer containing synapses in the json state. Default is 'synapses'
    Returns:
        pre_pt,post_pt,ctr_pt: list, lists of coordinates for synapses.'''
    
    
    syns = nglui.parser.line_annotations(state,synapse_layer)
    if np.shape(syns)[0] != 2:
        raise Exception( print('Incorrectly formatted synapse annotation. Requires two lists: Presynapse coordinates, Postsynapse coordinates'))
           
    else:
        pre_pt = syns[0]
        post_pt = syns[1]
        ctr_pt = (np.array(pre_pt) + np.array(post_pt)) / 2
    return(pre_pt,post_pt,np.ndarray.tolist(ctr_pt))


def format_synapse(pre,post,ctr):
    ''' Format a synapse for upload to a synapse table
    Args:
           pre: list, mip0 xyz coordinates to presynapse
           post: list, mip0 xyz coordinates to postsynapse
           ctr: list, mip0 xyz coordinates to center point
    Returns:
           data: formatted dict for a synapse table annotation upload
    '''
    data = {
    "type": "synapse",
    'pre_pt': {'position': pre},
    'post_pt': {'position': post},
    'ctr_pt': {'position': ctr}

}
    return(data)

def upload_synapses(client,synapse_coordinates,table_name,description=None):
    ''' Add synapses to a synapse table.
    Args: 
        client: annotation client object
        synapse_coordinates: list, list of shape [3,n,3]. Dim 0 is [pre,post,ctr], Dim 1 is the entry, Dim 2 is [x,y,z]. Output of get_synapses
    ##TODO: Some sort of check for redundant synapses is necessary. We could make each upload need to go to a new table, but that seems absurd. 
    '''
    try:
        client.annotation.create_table(table_name=table_name,
                                   schema_name='synapse',
                                   description=description)
    except:
        print('table exists')
        
    for i in range(np.shape(synapse_coordinates)[1]):
        try:
            entry = format_synapse(synapse_coordinates[0][i],synapse_coordinates[1][i],synapse_coordinates[2][i])
            client.annotation.post_annotation(table_name=table_name, data=[entry])
        except:
            print('Fail',entry)

In [11]:
entries = soma_table_entries(state,layer_name='13B')

In [14]:
upload_cells(client,entries,'test',description='Testing sv id upload')

In [15]:
client.annotation.delete_table('test')

True

[{'tag': '13B_alpha1_T1R',
  'pt': {'position': [41119.5625, 93109.8125, 3264.537353515625]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha2_T1R',
  'pt': {'position': [44111.78125, 89631.3359375, 3265.533447265625]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha3_T1R',
  'pt': {'position': [44830.7578125, 90352.8828125, 3262.0146484375]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha4_T1R',
  'pt': {'position': [44813.73828125, 91078.640625, 3265.327392578125]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha5_T1R',
  'pt': {'position': [45531.21484375, 90928.9453125, 3350.559814453125]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha6_T1R',
  'pt': {'position': [45254.9921875, 93200.1015625, 3458.303466796875]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha7_T1R',
  'pt': {'position': [43109.46484375, 93467.203125, 3486.690673828125]},
  'superceded_id': [111111, 222222]},
 {'tag': '13B_alpha8_T1R',
  'pt': {'positi